<a href="https://colab.research.google.com/github/ldriverc/colab/blob/main/trafficanalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. INSTALACIÓN DE DEPENDENCIAS
!pip install pandas numpy matplotlib seaborn plotly scikit-learn
!pip install openpyxl xlrd
from google.colab import drive, files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import os
import glob


Aquí viene el código principal

In [ ]:
# 2. CONFIGURACIÓN Y CONEXIÓN A DRIVE
drive.mount('/content/drive')

class TrafficAnalyzer:
    def __init__(self, base_path='/content/drive/MyDrive/Mediciones'):
        self.base_path = base_path
        self.data = None
        self.peak_hours_morning = (7, 9)
        self.peak_hours_evening = (17, 20)
        self.scaler = StandardScaler()

    def load_all_excel_files(self):
        """Carga todos los archivos Excel de las carpetas especificadas"""
        print("🔄 Cargando archivos Excel...")

        all_data = []
        folders_processed = 0

        # Buscar recursivamente todos los archivos Excel
        for root, dirs, files in os.walk(self.base_path):
            excel_files = [f for f in files if f.endswith(('.xlsx', '.xls'))]

            if excel_files:
                print(f"📁 Procesando carpeta: {root}")
                folders_processed += 1

                for file in excel_files:
                    file_path = os.path.join(root, file)
                    try:
                        # Leer el archivo Excel
                        df = pd.read_excel(file_path)

                        # Agregar metadatos del archivo
                        df['archivo'] = file
                        df['carpeta'] = os.path.basename(root)
                        df['ruta_completa'] = file_path

                        all_data.append(df)
                        print(f"  ✅ {file}: {len(df)} registros")

                    except Exception as e:
                        print(f"  ❌ Error al leer {file}: {str(e)}")

        if all_data:
            self.data = pd.concat(all_data, ignore_index=True)
            print(f"\n🎉 Datos cargados exitosamente!")
            print(f"📊 Total de registros: {len(self.data)}")
            print(f"📁 Carpetas procesadas: {folders_processed}")

            # Preprocessing básico
            self._preprocess_data()
        else:
            print("❌ No se encontraron archivos Excel en la ruta especificada")

    def _preprocess_data(self):
        """Preprocesamiento de datos"""
        print("\n🔧 Preprocesando datos...")

        # ESTO ES MUY IMPORTANTE: Convertir columnas a tipos apropiados, sino no se leerá correctamente
        self.data['Fecha'] = pd.to_datetime(self.data['Fecha'], errors='coerce')

        # Manejar diferentes formatos de hora
        if self.data['Hora'].dtype == 'object':
            self.data['Hora'] = pd.to_datetime(self.data['Hora'], format='%H:%M:%S', errors='coerce').dt.time

        # Eliminar filas con valores NaT en 'Fecha' antes de combinar
        initial_rows = len(self.data)
        self.data.dropna(subset=['Fecha'], inplace=True)
        rows_dropped = initial_rows - len(self.data)
        if rows_dropped > 0:
            print(f"⚠️ Se eliminaron {rows_dropped} filas con valores NaT en 'Fecha'.")

        # Crear columna datetime combinada
        self.data['datetime'] = pd.to_datetime(
            self.data['Fecha'].astype(str) + ' ' + self.data['Hora'].astype(str)
        )

        # Extraer características temporales
        self.data['año'] = self.data['Fecha'].dt.year
        self.data['mes'] = self.data['Fecha'].dt.month
        self.data['dia'] = self.data['Fecha'].dt.day
        self.data['hora_num'] = self.data['datetime'].dt.hour
        self.data['minuto'] = self.data['datetime'].dt.minute
        self.data['dia_semana'] = self.data['Fecha'].dt.dayofweek
        self.data['es_fin_semana'] = self.data['dia_semana'].isin([5, 6])

        # Identificar horas punta
        self.data['es_hora_punta'] = (
            ((self.data['hora_num'] >= self.peak_hours_morning[0]) &
             (self.data['hora_num'] < self.peak_hours_morning[1])) |
            ((self.data['hora_num'] >= self.peak_hours_evening[0]) &
             (self.data['hora_num'] < self.peak_hours_evening[1]))
        )

        # Limpiar datos atípicos con Isolation Forest
        self._detect_outliers()

        print(f"✅ Preprocesamiento completado")
        print(f"📅 Rango de fechas: {self.data['Fecha'].min()} - {self.data['Fecha'].max()}")
        print(f"🚗 Pistas únicas: {self.data['Pista'].nunique()}")

    def _detect_outliers(self):
        """Detecta y marca outliers usando Isolation Forest"""
        print("🔍 Detectando valores atípicos...")

        # Aplicar Isolation Forest
        iso_forest = IsolationForest(contamination=0.1, random_state=42)
        features = ['Flujo', 'Ocupación', 'hora_num', 'Pista']

        # Handle potential non-numeric 'Pista' values or missing values
        temp_data = self.data.copy()
        temp_data['Pista'] = pd.to_numeric(temp_data['Pista'], errors='coerce')
        temp_data.dropna(subset=features, inplace=True)


        if not temp_data.empty:
            outliers = iso_forest.fit_predict(temp_data[features])
            # Align outliers back to the original dataframe index
            self.data['es_outlier'] = False # Initialize all as not outlier
            self.data.loc[temp_data.index, 'es_outlier'] = outliers == -1
        else:
            print("⚠️ No hay datos válidos para la detección de outliers.")
            self.data['es_outlier'] = False


        outlier_count = self.data['es_outlier'].sum()
        print(f"🚨 Outliers detectados: {outlier_count} ({outlier_count/len(self.data)*100:.2f}%)")


    def analyze_peak_hours_comparison(self):
        """Análisis comparativo de horas punta entre 2024 y 2025"""
        print("\n📈 ANÁLISIS COMPARATIVO HORAS PUNTA 2024 vs 2025")
        print("=" * 60)

        # Filtrar datos de horas punta sin outliers
        peak_data = self.data[
            (self.data['es_hora_punta'] == True) &
            (self.data['es_outlier'] == False) &
            (self.data['año'].isin([2024, 2025]))
        ]

        if len(peak_data) == 0:
            print("❌ No hay datos de horas punta para 2024 y 2025")
            return

        # Análisis por año
        results = {}
        for year in [2024, 2025]:
            year_data = peak_data[peak_data['año'] == year]

            if len(year_data) > 0:
                results[year] = {
                    'flujo_promedio': year_data['Flujo'].mean(),
                    'ocupacion_promedio': year_data['Ocupación'].mean(),
                    'flujo_std': year_data['Flujo'].std(),
                    'ocupacion_std': year_data['Ocupación'].std(),
                    'registros': len(year_data),
                    'pistas_unicas': year_data['Pista'].nunique()
                }

        # Mostrar resultados
        for year, stats in results.items():
            print(f"\n📊 AÑO {year}:")
            print(f"   🚗 Flujo promedio: {stats['flujo_promedio']:.2f} ± {stats['flujo_std']:.2f}")
            print(f"   ⏱️  Ocupación promedio: {stats['ocupacion_promedio']:.2f} ± {stats['ocupacion_std']:.2f} ms")
            print(f"   📋 Registros: {stats['registros']}")
            print(f"   🛣️  Pistas únicas: {stats['pistas_unicas']}")

        # Análisis de cambios
        if 2024 in results and 2025 in results:
            print(f"\n🔄 CAMBIOS 2024 → 2025:")

            flujo_change = ((results[2025]['flujo_promedio'] - results[2024]['flujo_promedio']) /
                           results[2024]['flujo_promedio'] * 100)
            ocupacion_change = ((results[2025]['ocupacion_promedio'] - results[2024]['ocupacion_promedio']) /
                               results[2024]['ocupacion_promedio'] * 100)

            print(f"   🚗 Cambio en flujo: {flujo_change:+.2f}%")
            print(f"   ⏱️  Cambio en ocupación: {ocupacion_change:+.2f}%")

            # Interpretación
            if flujo_change > 5:
                print("   📈 AUMENTO SIGNIFICATIVO en el flujo vehicular")
            elif flujo_change < -5:
                print("   📉 DISMINUCIÓN SIGNIFICATIVA en el flujo vehicular")
            else:
                print("   ➡️  Flujo vehicular ESTABLE")

        return results

    def ml_traffic_prediction(self):
        """Modelo de ML para predicción de tráfico"""
        print("\n🤖 MODELO DE MACHINE LEARNING PARA PREDICCIÓN")
        print("=" * 60)

        # Preparar datos para ML
        ml_data = self.data[self.data['es_outlier'] == False].copy()

        # Features engineering
        features = [
            'hora_num', 'minuto', 'dia_semana', 'mes', 'dia',
            'Pista', 'es_hora_punta', 'es_fin_semana'
        ]

        # Ensure 'Pista' is numeric before one-hot encoding
        ml_data['Pista'] = pd.to_numeric(ml_data['Pista'], errors='coerce')
        ml_data.dropna(subset=['Pista'], inplace=True)


        # Crear variables dummy para categóricas
        ml_data_encoded = pd.get_dummies(ml_data, columns=['Pista'], prefix='pista')

        # Actualizar features con las nuevas columnas
        pista_cols = [col for col in ml_data_encoded.columns if col.startswith('pista_')]
        features_final = ['hora_num', 'minuto', 'dia_semana', 'mes', 'dia',
                         'es_hora_punta', 'es_fin_semana'] + pista_cols

        # Ensure all features exist and are numeric
        for col in features_final:
            if col not in ml_data_encoded.columns:
                ml_data_encoded[col] = 0
            else:
                ml_data_encoded[col] = pd.to_numeric(ml_data_encoded[col], errors='coerce')

        ml_data_encoded.dropna(subset=features_final, inplace=True)


        X = ml_data_encoded[features_final]
        y_flujo = ml_data_encoded['Flujo']
        y_ocupacion = ml_data_encoded['Ocupación']

        # Check if there's enough data after cleaning
        if len(X) == 0:
            print("❌ No hay suficientes datos válidos para entrenar el modelo de ML.")
            return None, None, None


        # Dividir datos
        X_train, X_test, y_flujo_train, y_flujo_test, y_ocupacion_train, y_ocupacion_test = train_test_split(
            X, y_flujo, y_ocupacion, test_size=0.2, random_state=42
        )

        # Entrenar modelos
        print("🎯 Entrenando modelos Random Forest...")

        # Modelo para flujo
        rf_flujo = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_flujo.fit(X_train, y_flujo_train)

        # Modelo para ocupación
        rf_ocupacion = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_ocupacion.fit(X_train, y_ocupacion_train)

        # Predicciones
        y_flujo_pred = rf_flujo.predict(X_test)
        y_ocupacion_pred = rf_ocupacion.predict(X_test)

        # Métricas
        print(f"\n📊 MÉTRICAS DEL MODELO:")
        print(f"   🚗 FLUJO:")
        print(f"      R²: {r2_score(y_flujo_test, y_flujo_pred):.4f}")
        print(f"      MAE: {mean_absolute_error(y_flujo_test, y_flujo_pred):.4f}")
        print(f"      RMSE: {np.sqrt(mean_squared_error(y_flujo_test, y_flujo_pred)):.4f}")

        print(f"   ⏱️  OCUPACIÓN:")
        print(f"      R²: {r2_score(y_ocupacion_test, y_ocupacion_pred):.4f}")
        print(f"      MAE: {mean_absolute_error(y_ocupacion_test, y_ocupacion_pred):.4f}")
        print(f"      RMSE: {np.sqrt(mean_squared_error(y_ocupacion_test, y_ocupacion_pred)):.4f}")

        # Importancia de features
        self._plot_feature_importance(rf_flujo, features_final, "Flujo")
        self._plot_feature_importance(rf_ocupacion, features_final, "Ocupación")

        return rf_flujo, rf_ocupacion, features_final

    def _plot_feature_importance(self, model, features, target_name):
        """Visualiza la importancia de las características"""
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]

        plt.figure(figsize=(12, 6))
        plt.title(f'Importancia de Características - {target_name}')
        plt.bar(range(len(importances)), importances[indices])
        plt.xticks(range(len(importances)), [features[i] for i in indices], rotation=45)
        plt.tight_layout()
        plt.show()

    def clustering_analysis(self):
        """Análisis de clustering para identificar patrones"""
        print("\n🔍 ANÁLISIS DE CLUSTERING - PATRONES DE TRÁFICO")
        print("=" * 60)

        # Preparar datos para clustering
        cluster_data = self.data[self.data['es_outlier'] == False].copy()

        # Ensure 'Pista' is numeric before grouping
        cluster_data['Pista'] = pd.to_numeric(cluster_data['Pista'], errors='coerce')
        cluster_data.dropna(subset=['Pista'], inplace=True)


        # Agrupar por hora y calcular promedios
        hourly_patterns = cluster_data.groupby(['hora_num', 'Pista']).agg({
            'Flujo': 'mean',
            'Ocupación': 'mean'
        }).reset_index()

        if hourly_patterns.empty:
             print("❌ No hay suficientes datos válidos para el análisis de clustering.")
             return None, None


        # Aplicar K-means
        features_cluster = ['hora_num', 'Flujo', 'Ocupación']
        X_cluster = hourly_patterns[features_cluster]

        # Normalizar datos
        X_cluster_scaled = self.scaler.fit_transform(X_cluster)

        # Encontrar número óptimo de clusters (método del codo)
        inertias = []
        K_range = range(2, min(10, len(X_cluster_scaled) + 1)) # Adjust K_range based on data size


        for k in K_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10) # Add n_init
            kmeans.fit(X_cluster_scaled)
            inertias.append(kmeans.inertia_)

        # Aplicar clustering con k óptimo (asumimos k=4 para este ejemplo)
        optimal_k = 4
        if len(X_cluster_scaled) < optimal_k:
             print(f"⚠️ No hay suficientes puntos de datos para {optimal_k} clusters. Usando {len(X_cluster_scaled)} clusters.")
             optimal_k = len(X_cluster_scaled)
             if optimal_k < 2:
                 print("❌ No hay suficientes datos para realizar clustering.")
                 return None, None

        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10) # Add n_init
        hourly_patterns['cluster'] = kmeans.fit_predict(X_cluster_scaled)

        # Visualizar clusters
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Análisis de Clustering - Patrones de Tráfico', fontsize=16)

        # Método del codo
        axes[0, 0].plot(K_range, inertias, 'bo-')
        axes[0, 0].set_xlabel('Número de Clusters')
        axes[0, 0].set_ylabel('Inercia')
        axes[0, 0].set_title('Método del Codo')
        axes[0, 0].grid(True)

        # Scatter plot de clusters
        scatter = axes[0, 1].scatter(hourly_patterns['Flujo'], hourly_patterns['Ocupación'],
                                    c=hourly_patterns['cluster'], cmap='viridis')
        axes[0, 1].set_xlabel('Flujo Promedio')
        axes[0, 1].set_ylabel('Ocupación Promedio')
        axes[0, 1].set_title('Clusters de Tráfico')
        plt.colorbar(scatter, ax=axes[0, 1])

        # Patrones por hora
        for cluster in range(optimal_k):
            cluster_data_plot = hourly_patterns[hourly_patterns['cluster'] == cluster]
            axes[1, 0].plot(cluster_data_plot['hora_num'], cluster_data_plot['Flujo'],
                           label=f'Cluster {cluster}', marker='o')

        axes[1, 0].set_xlabel('Hora del día')
        axes[1, 0].set_ylabel('Flujo Promedio')
        axes[1, 0].set_title('Patrones de Flujo por Cluster')
        axes[1, 0].legend()
        axes[1, 0].grid(True)

        # Distribución de clusters
        cluster_counts = hourly_patterns['cluster'].value_counts().sort_index()
        axes[1, 1].bar(cluster_counts.index, cluster_counts.values)
        axes[1, 1].set_xlabel('Cluster')
        axes[1, 1].set_ylabel('Cantidad de Puntos')
        axes[1, 1].set_title('Distribución de Clusters')

        plt.tight_layout()
        plt.show()

        # Análisis de características de cada cluster
        print("\n📋 CARACTERÍSTICAS DE CADA CLUSTER:")
        for cluster in range(optimal_k):
            cluster_data = hourly_patterns[hourly_patterns['cluster'] == cluster]
            print(f"\n🏷️  CLUSTER {cluster}:")
            print(f"   📊 Puntos: {len(cluster_data)}")
            print(f"   🚗 Flujo promedio: {cluster_data['Flujo'].mean():.2f}")
            print(f"   ⏱️  Ocupación promedio: {cluster_data['Ocupación'].mean():.2f}")
            print(f"   🕐 Horas típicas: {cluster_data['hora_num'].min()}-{cluster_data['hora_num'].max()}")

        return hourly_patterns, kmeans

    def generate_comprehensive_report(self):
        """Genera un reporte completo con visualizaciones"""
        print("\n📊 GENERANDO REPORTE COMPLETO")
        print("=" * 60)

        # Resumen general
        print(f"📋 RESUMEN GENERAL:")
        print(f"   📅 Período: {self.data['Fecha'].min()} - {self.data['Fecha'].max()}")
        print(f"   📊 Total registros: {len(self.data)}")
        print(f"   🛣️  Pistas analizadas: {self.data['Pista'].nunique()}")
        print(f"   🚨 Outliers detectados: {self.data['es_outlier'].sum()}")

        # Estadísticas por año
        yearly_stats = self.data.groupby('año').agg({
            'Flujo': ['mean', 'std', 'min', 'max'],
            'Ocupación': ['mean', 'std', 'min', 'max']
        }).round(2)

        print(f"\n📈 ESTADÍSTICAS POR AÑO:")
        print(yearly_stats)

        # Crear visualizaciones
        self._create_visualizations()

    def _create_visualizations(self):
        """Crea visualizaciones comprehensivas"""

        # Configurar el estilo
        plt.style.use('seaborn-v0_8')

        # 1. Distribución temporal del tráfico
        fig, axes = plt.subplots(2, 2, figsize=(20, 15))
        fig.suptitle('Análisis Temporal del Tráfico Vehicular', fontsize=16)

        # Flujo por hora del día
        hourly_flow = self.data.groupby('hora_num')['Flujo'].mean()
        axes[0, 0].plot(hourly_flow.index, hourly_flow.values, marker='o', linewidth=2)
        axes[0, 0].axvspan(7, 9, alpha=0.3, color='red', label='Hora punta mañana')
        axes[0, 0].axvspan(17, 20, alpha=0.3, color='red', label='Hora punta tarde')
        axes[0, 0].set_xlabel('Hora del día')
        axes[0, 0].set_ylabel('Flujo promedio')
        axes[0, 0].set_title('Flujo vehicular por hora del día')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # Ocupación por hora del día
        hourly_occ = self.data.groupby('hora_num')['Ocupación'].mean()
        axes[0, 1].plot(hourly_occ.index, hourly_occ.values, marker='s', linewidth=2, color='orange')
        axes[0, 1].axvspan(7, 9, alpha=0.3, color='red')
        axes[0, 1].axvspan(17, 20, alpha=0.3, color='red')
        axes[0, 1].set_xlabel('Hora del día')
        axes[0, 1].set_ylabel('Ocupación promedio (ms)')
        axes[0, 1].set_title('Ocupación por hora del día')
        axes[0, 1].grid(True, alpha=0.3)

        # Flujo por día de la semana
        daily_flow = self.data.groupby('dia_semana')['Flujo'].mean()
        days = ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom']
        axes[1, 0].bar(daily_flow.index, daily_flow.values, color='skyblue') # Use index for positions
        axes[1, 0].set_xticks(daily_flow.index) # Set tick positions
        axes[1, 0].set_xticklabels(days) # Set tick labels
        axes[1, 0].set_xlabel('Día de la semana')
        axes[1, 0].set_ylabel('Flujo promedio')
        axes[1, 0].set_title('Flujo vehicular por día de la semana')
        axes[1, 0].grid(True, alpha=0.3)

        # Comparación por pista
        pista_stats = self.data.groupby('Pista')['Flujo'].mean().sort_values(ascending=False)
        axes[1, 1].bar(pista_stats.index.astype(str), pista_stats.values, color='lightgreen')
        axes[1, 1].set_xlabel('Pista')
        axes[1, 1].set_ylabel('Flujo promedio')
        axes[1, 1].set_title('Flujo promedio por pista')
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

        # 2. Heatmap de tráfico por hora y día
        pivot_data = self.data.pivot_table(
            values='Flujo',
            index='hora_num',
            columns='dia_semana',
            aggfunc='mean'
        )

        plt.figure(figsize=(12, 8))
        sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='YlOrRd',
                   xticklabels=days, yticklabels=range(24))
        plt.title('Heatmap: Flujo vehicular por hora y día de la semana')
        plt.xlabel('Día de la semana')
        plt.ylabel('Hora del día')
        plt.tight_layout()
        plt.show()

Mounted at /content/drive


In [ ]:
# 3. FUNCIÓN PRINCIPAL DE EJECUCIÓN
def main():
    """Función principal que ejecuta todo el análisis"""
    print("🚀 INICIANDO ANÁLISIS DE TRÁFICO VEHICULAR CON ML")
    print("=" * 70)

    # Inicializar analizador
    analyzer = TrafficAnalyzer()

    # Cargar datos
    analyzer.load_all_excel_files()

    if analyzer.data is None:
        print("❌ No se pudieron cargar los datos. Verifica la ruta.")
        return

    # Ejecutar análisis
    print("\n🔍 EJECUTANDO ANÁLISIS...")

    # 1. Análisis comparativo básico
    comparison_results = analyzer.analyze_peak_hours_comparison()

    # 2. Modelo de ML
    rf_flujo, rf_ocupacion, features = analyzer.ml_traffic_prediction()

    # 3. Análisis de clustering
    patterns, kmeans_model = analyzer.clustering_analysis()

    # 4. Reporte completo
    analyzer.generate_comprehensive_report()

    print("\n✅ ANÁLISIS COMPLETADO")
    print("📊 Todos los resultados han sido generados exitosamente")

    return analyzer, rf_flujo, rf_ocupacion, patterns, kmeans_model

In [ ]:
# 4. FUNCIÓN DE PREDICCIÓN PARA NUEVOS DATOS
def predict_traffic(analyzer, rf_flujo, rf_ocupacion, features, hora, pista, dia_semana, mes=1):
    """Función para predecir tráfico en condiciones específicas"""
    # Crear DataFrame con las características
    new_data = pd.DataFrame({
        'hora_num': [hora],
        'minuto': [0],
        'dia_semana': [dia_semana],
        'mes': [mes],
        'dia': [1],
        'es_hora_punta': [hora in range(7, 9) or hora in range(17, 20)],
        'es_fin_semana': [dia_semana in [5, 6]]
    })

    # Agregar columnas de pista (one-hot encoding)
    for feature in features:
        if feature.startswith('pista_'):
            pista_num = feature.split('_')[1]
            new_data[feature] = [1 if pista_num == str(pista) else 0]

    # Asegurar que tenemos todas las columnas necesarias
    for feature in features:
        if feature not in new_data.columns:
            new_data[feature] = [0]

    # Ordenar columnas según el orden de entrenamiento
    new_data = new_data[features]

    # Realizar predicción
    flujo_pred = rf_flujo.predict(new_data)[0]
    ocupacion_pred = rf_ocupacion.predict(new_data)[0]

    return flujo_pred, ocupacion_pred

In [ ]:
# 5. EJECUTAR ANÁLISIS
if __name__ == "__main__":
    # Ejecutar análisis principal
    analyzer, rf_flujo, rf_ocupacion, patterns, kmeans_model = main()

    # Ejemplo de predicción
    print("\n🔮 EJEMPLO DE PREDICCIÓN:")
    flujo_pred, ocupacion_pred = predict_traffic(
        analyzer, rf_flujo, rf_ocupacion,
        analyzer.ml_traffic_prediction()[2],  # features
        hora=8,  # 8 AM
        pista=1,
        dia_semana=1,  # Martes
        mes=7  # Julio
    )

    print(f"📊 Predicción para Pista 1, Martes 8:00 AM:")
    print(f"   🚗 Flujo estimado: {flujo_pred:.2f} vehículos")
    print(f"   ⏱️  Ocupación estimada: {ocupacion_pred:.2f} ms")

In [ ]:
# 5. EJECUTAR ANÁLISIS
if __name__ == "__main__":
    # Ejecutar análisis principal
    analyzer, rf_flujo, rf_ocupacion, patterns, kmeans_model = main()

    # Ejemplo de predicción
    print("\n🔮 EJEMPLO DE PREDICCIÓN:")
    # Ensure features are passed correctly
    flujo_pred, ocupacion_pred = predict_traffic(
        analyzer, rf_flujo, rf_ocupacion,
        analyzer.ml_traffic_prediction()[2],  # features
        hora=8,  # 8 AM
        pista=1,
        dia_semana=1,  # Martes
        mes=7  # Julio
    )

    print(f"📊 Predicción para Pista 1, Martes 8:00 AM:")
    print(f"   🚗 Flujo estimado: {flujo_pred:.2f} vehículos")
    print(f"   ⏱️  Ocupación estimada: {ocupacion_pred:.2f} ms")